In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
train_url = '/home/thinker/NLP/NLP/FromScratch/error detection/data/data/train_data.csv'
test_url = '/home/thinker/NLP/NLP/FromScratch/error detection/data/data/test_data.csv'
df_train = pd.read_csv(train_url)
df_test = pd.read_csv(test_url)
stop_words_df = pd.read_excel('/home/thinker/NLP/NLP/FromScratch/error detection/data/stopwords_bangla.xlsx',index_col=False)

In [2]:
STOPWORDS = set([word.strip() for word in stop_words_df['words']])

In [3]:
import re
def preprocess(x):
    html_pattern = re.compile('<.*?>')
    x = html_pattern.sub(r'', x)
    x = " ".join([word for word in str(x).split() if word not in STOPWORDS])
    return x
df_train['Comment'] = df_train['Comment'].apply(lambda x: preprocess(x))
df_test['Comment'] = df_test['Comment'].apply(lambda x:preprocess(x))

In [4]:
def co(x):
    d = ['Code Switching','Spelling','Multiple Errors','Grammatical']
    s = set(d)
    if x not in s:
        return "Correct"
    else:
        return x
df_train['Category'] = df_train['Category'].apply(lambda x:co(x))
df_test['Category'] = df_test['Category'].apply(lambda x:co(x))

In [5]:
df_train

,Video Title,Genre,Comment,Error,Category,Correct form
0,"ওবায়দুল কাদের বললেন, ‘খেলা হবে’; আর রুমিন ফারহ...",Politics,কাদের খেলব কাদের খেলতেই না,1,Grammatical,কাদের কি খেলবে কাদের তো খেলতেই পারে না
1,পুলিশের গাড়ির ওপর চড়াও বিএনপির কর্মীরা | BN...,Politics,এসব আরো ঠাসা,1,Spelling,এসব করে আরো কোণঠাসা হবে
2,Ayub Bachchu | Ek Akash Tara | আইয়ুব বাচ্চু |...,Entertainment,যুগ যুগ গেথে গান,0,Correct,NaN
3,যে প্রেম কাহিনী কোন বাধা মানেনি | BBC Bangla,Miscellaneous,অাচছা অাপু এলাজী থাকলে টিকা জাবেনা,1,Spelling,আচ্ছা আপু এলার্জী থাকলে টিকা নেওয়া যাবেনা
4,তুরস্কের চেয়ে ভয়াবহ ভূমিকম্পের ঝুঁকিতে বাংলাদে...,News,হে আল্লাহ জালিমদের সন্তান সন্তদের হেফাজত,0,Correct,NaN
...,...,...,...,...,...,...
8031,Shitom Ahmed - Chorabali (Lyrics) || কেন লাগে ...,Entertainment,সত্যি শুন্য লাগে,0,Correct,NaN
8032,হৃদয় ছুঁয়ে যাওয়া ৭টি সেরা ইমোশনাল বিজ্ঞাপন ...,Entertainment,বিজ্ঞাপন গুলো চোখে পানি আসলো ভাই,0,Correct,NaN
8033,আইপিএলের নিলাম তালিকায় পাঁচ বাংলাদেশি | IPL | ...,Sports,টাকা সবদিক,1,Spelling,একবার যখন টাকা হয় তখন সব দিক দিয়ে আসে
8034,মাহমুদুল্লাহর সেরা ১০টি ইনিংস || 10 Greatest I...,Sports,অসাধারণ ইউটিউব ভিডিও সাথে চমৎকার ব্যাকগ্রাউন্ড,0,Correct,NaN


In [6]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(df_train.Category)
Test_Y = Encoder.fit_transform(df_test.Category)

In [7]:
df_all  = pd.concat([df_train, df_test], ignore_index=True)
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(df_all['Comment'])
Train_X_Tfidf = Tfidf_vect.transform(df_train['Comment'])
Test_X_Tfidf = Tfidf_vect.transform(df_test['Comment'])

In [8]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.8209    0.3481    0.4889       158
           1     0.6096    0.9473    0.7418      1157
           2     0.5625    0.0703    0.1250       128
           3     0.5000    0.0145    0.0282        69
           4     0.4844    0.1242    0.1978       499

    accuracy                         0.6082      2011
   macro avg     0.5955    0.3009    0.3163      2011
weighted avg     0.5884    0.6082    0.5232      2011

